In [1]:
polling_period_in_seconds = 1
monitoring_duration_in_seconds_this_notebook=1
monitoring_duration_in_seconds_other_notebooks=600

In [2]:
!hostname

ff9b09e120ce


In [3]:
!jupyter --version

4.4.0


https://stackoverflow.com/questions/276052/how-to-get-current-cpu-and-ram-usage-in-python

https://pypi.org/project/psutil/<BR>
https://psutil.readthedocs.io/en/latest/

potential additional work:
* add strace capture
 * https://github.com/blapid/strace2py
 * https://python-ptrace.readthedocs.io/en/latest/syscall.html
 * https://github.com/dirtyharrycallahan/pystrace

In [4]:
!pip install psutil

In [5]:
import requests
import json
import subprocess
import psutil
import pickle
import os
import time
import datetime
import pandas
print('pandas',pandas.__version__)
import matplotlib.pyplot as plt

pandas 0.23.4


# one-time system details

In [6]:
print(psutil.cpu_freq())

None


In [7]:
pid = os.getpid()
print(pid)

798


In [8]:
py = psutil.Process(pid)
print(py)

psutil.Process(pid=798, name='python', started='12:45:14')


In [9]:
py.name()

'python'

In [10]:
py.exe()

'/opt/conda/bin/python3.6'

In [11]:
py.cwd()

'/home/jovyan/week_09_timeseries/host_streaming_stats'

In [12]:
py.cmdline()

['/opt/conda/bin/python',
 '-m',
 'ipykernel_launcher',
 '-f',
 '/home/jovyan/.local/share/jupyter/runtime/kernel-cdff474c-25b7-4ed0-bc65-7b6ba587d83e.json']

# system performance

In [13]:
# gives a single float value
psutil.cpu_percent(percpu=True)

[44.7, 34.6]

In [14]:
psutil.cpu_times_percent(percpu=True)

[scputimes(user=22.8, nice=0.0, system=17.9, idle=31.7, iowait=22.1, irq=0.0, softirq=5.5, steal=0.0, guest=0.0, guest_nice=0.0),
 scputimes(user=19.0, nice=0.0, system=13.9, idle=55.7, iowait=8.9, irq=0.0, softirq=2.5, steal=0.0, guest=0.0, guest_nice=0.0)]

In [15]:
for reslt in psutil.cpu_times_percent(percpu=True):
    print(dict(reslt._asdict()))

{'user': 2.0, 'nice': 0.0, 'system': 0.0, 'idle': 0.0, 'iowait': 0.0, 'irq': 0.0, 'softirq': 0.0, 'steal': 0.0, 'guest': 0.0, 'guest_nice': 0.0}
{'user': 0.0, 'nice': 0.0, 'system': 3.0, 'idle': 0.0, 'iowait': 0.0, 'irq': 0.0, 'softirq': 0.0, 'steal': 0.0, 'guest': 0.0, 'guest_nice': 0.0}


In [16]:
# gives an object with many fields
psutil.virtual_memory()

svmem(total=2096164864, available=1720184832, percent=17.9, used=229212160, free=1699770368, active=197160960, inactive=128303104, buffers=17334272, cached=149848064, shared=4096, slab=44593152)

In [17]:
# you can convert that object to a dictionary 
dict(psutil.virtual_memory()._asdict())

{'total': 2096164864,
 'available': 1720172544,
 'percent': 17.9,
 'used': 229216256,
 'free': 1699770368,
 'active': 197165056,
 'inactive': 128290816,
 'buffers': 17334272,
 'cached': 149843968,
 'shared': 4096,
 'slab': 44593152}

In [18]:
dict(psutil.swap_memory()._asdict())

{'total': 1073737728,
 'used': 232820736,
 'free': 840916992,
 'percent': 21.7,
 'sin': 1905643520,
 'sout': 6887395328}

In [19]:
dict(psutil.cpu_stats()._asdict())

{'ctx_switches': 27523344,
 'interrupts': 12341847,
 'soft_interrupts': 4016104,
 'syscalls': 0}

In [20]:
psutil.disk_io_counters(perdisk=True).keys()

dict_keys(['loop0', 'loop1', 'loop2', 'loop3', 'loop4', 'loop5', 'loop6', 'loop7', 'nbd0', 'nbd1', 'nbd2', 'nbd3', 'nbd4', 'nbd5', 'nbd6', 'nbd7', 'nbd8', 'nbd9', 'nbd10', 'nbd11', 'nbd12', 'nbd13', 'nbd14', 'nbd15', 'sda', 'sda1', 'sr0', 'sr1', 'sr2'])

In [21]:
dict(psutil.disk_io_counters(perdisk=True)['loop0']._asdict())

{'read_count': 0,
 'write_count': 0,
 'read_bytes': 0,
 'write_bytes': 0,
 'read_time': 0,
 'write_time': 0,
 'read_merged_count': 0,
 'write_merged_count': 0,
 'busy_time': 0}

In [22]:
psutil.sensors_temperatures()

{}

In [23]:
psutil.sensors_fans()

{}

In [24]:
psutil.sensors_battery()

### put everything into a single row

In [25]:
def row_for_system_performance(row):
    if not isinstance(row,dict):
        raise TypeError("ERROR: expected a dict as input, got",type(row))
        return None
    cpu_pct_list = psutil.cpu_percent(percpu=True)
    for indx in range(len(cpu_pct_list)):
        row['CPU percent for '+str(indx)]=cpu_pct_list[indx]
    
    row['time in unix seconds']=time.time()
    row['datetime']=datetime.datetime.utcfromtimestamp(row['time in unix seconds'])

    indx=0
    for reslt in psutil.cpu_times_percent(percpu=True):
        indx+=1
        for k,v in dict(reslt._asdict()).items():
            row['CPU '+str(indx)+' times '+k]=v
    for disk_label in psutil.disk_io_counters(perdisk=True).keys():
        for k,v in dict(psutil.disk_io_counters(perdisk=True)[disk_label]._asdict()).items():
            row['disk '+disk_label+' '+k]=v
    for k,v in dict(psutil.cpu_stats()._asdict()).items():
        row['CPU stats '+k]=v
    for k,v in dict(psutil.virtual_memory()._asdict()).items():
        row['system virtual memory '+k]=v
    for k,v in dict(psutil.swap_memory()._asdict()).items():
        row['system swap memory '+k]=v
    return row

# performance of current process

In [26]:
os.getpid()

798

In [27]:
type(os.getpid())

int

In [28]:
py = psutil.Process(os.getpid())
print(py)

psutil.Process(pid=798, name='python', started='12:45:14')


In [29]:
dict(py.memory_info()._asdict())

{'rss': 110751744,
 'vms': 854990848,
 'shared': 31674368,
 'text': 2232320,
 'lib': 0,
 'data': 189763584,
 'dirty': 0}

In [30]:
memoryUse = py.memory_info()[0]/2.**30  # memory use in GB...I think
print('memory use:', memoryUse,'GB')

memory use: 0.10314559936523438 GB


In [31]:
dict(py.memory_full_info()._asdict())

{'rss': 110751744,
 'vms': 854990848,
 'shared': 31674368,
 'text': 2232320,
 'lib': 0,
 'data': 189763584,
 'dirty': 0,
 'uss': 95760384,
 'pss': 102096896,
 'swap': 0}

In [32]:
py.cpu_percent()

0.0

In [33]:
dict(py.cpu_times()._asdict())

{'user': 1.9, 'system': 1.09, 'children_user': 2.34, 'children_system': 1.53}

In [34]:
py.cmdline()

['/opt/conda/bin/python',
 '-m',
 'ipykernel_launcher',
 '-f',
 '/home/jovyan/.local/share/jupyter/runtime/kernel-cdff474c-25b7-4ed0-bc65-7b6ba587d83e.json']

In [35]:
dict(py.num_ctx_switches()._asdict())

{'voluntary': 1759, 'involuntary': 3103}

In [36]:
dict(py.io_counters()._asdict())

{'read_count': 6566,
 'write_count': 1260,
 'read_bytes': 46297088,
 'write_bytes': 4096,
 'read_chars': 29429290,
 'write_chars': 924937}

In [37]:
def row_for_performance_per_pid(row,pid):
    if not isinstance(row,dict):
        raise TypeError("ERROR: expected a dict as input for row, got",type(row))
        return None
    if not isinstance(pid,int):
        raise TypeError("ERROR: expected a int as input for pid, got",type(pid))
        return None
    
    #if psutil.Process(pid).is_running():
    for p in psutil.process_iter():
        if (pid==p.pid):
            try:
                mem_dict=dict(psutil.Process(pid).memory_full_info()._asdict())
            except psutil.NoSuchProcess:
                return row
            for k,v in mem_dict.items():
                row['process memory: '+k]=v

    try:
        cpu_times_dict=dict(psutil.Process(pid).cpu_times()._asdict())
    except psutil.NoSuchProcess:
        return row
    for k,v in cpu_times_dict.items():
        row['process cpu times: '+k]=v

    try:
        context_switches_dict=dict(psutil.Process(pid).num_ctx_switches()._asdict())
    except psutil.NoSuchProcess:
        return row        
    for k,v in context_switches_dict.items():
        row['process number context switches: '+k]=v

    try:
        io_counter_dict = dict(psutil.Process(pid).io_counters()._asdict())
    except psutil.NoSuchProcess:
        return row
    for k,v in io_counter_dict.items():
        row['process io counters: '+k]=v
    return row

# gather data and create plots for this notebook

In [38]:
list_of_rows=[]
start_time = time.time()
for indx in range(monitoring_duration_in_seconds_this_notebook):
    row = {}
    row = row_for_system_performance(row)
    row = row_for_performance_per_pid(row,os.getpid())
    list_of_rows.append(row)
    time.sleep(polling_period_in_seconds)

print('elapsed time:',time.time()-start_time,'seconds')

df = pandas.DataFrame(list_of_rows)

elapsed time: 1.035003662109375 seconds


In [39]:
print(df.shape)
df.head()

(1, 328)


,CPU 1 times guest,CPU 1 times guest_nice,CPU 1 times idle,CPU 1 times iowait,CPU 1 times irq,CPU 1 times nice,CPU 1 times softirq,CPU 1 times steal,CPU 1 times system,CPU 1 times user,...,system virtual memory buffers,system virtual memory cached,system virtual memory free,system virtual memory inactive,system virtual memory percent,system virtual memory shared,system virtual memory slab,system virtual memory total,system virtual memory used,time in unix seconds
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,32.0,29.0,...,17334272,149848064,1698508800,128303104,18.0,4096,44642304,2096164864,230473728,1.553432e+09


In [40]:
df.to_pickle("single_process_df_this_notebook_"+str(time.time())+".pkl")

#df = pandas.read_pickle("single_process_df_this_notebook_1552783848.2168732.pkl")

In [41]:
def scatter_plot_column(df,col_name):
    if not isinstance(df,pandas.core.frame.DataFrame):
        raise TypeError('ERROR: expected input of type dataframe, got',type(df))
        return None
    if not isinstance(col_name,str):
        raise TypeError('ERROR: expected input of type str, got',type(col_name))
        return None
    plt.plot_date(x=df['datetime'],y=df[col_name])
    plt.xticks(rotation=40)
    _=plt.ylabel(col_name,fontsize=14)
    _=plt.xlabel('time',fontsize=14)
    plt.show()
    return

# track other notebooks

Plan: associate pid with each notebook kernel ID, then track the stats for that notebook by pid

works even when the kernel of the other notebook is restarted

detects when new newbook kernels being added

https://github.com/iiSeymour/nbtop/blob/master/nbtop/main.py

### get a list of all active sessions

In [42]:
r = requests.get('http://localhost:8888/api/sessions')
r.status_code

200

In [43]:
list_of_kernel_dicts = r.json()
list_of_kernel_dicts[0]

{'id': '634f6b06-0d01-4880-8165-8508ac87fe6a',
 'path': 'week_09_timeseries/host_streaming_stats/host_cpu_and_memory.ipynb',
 'name': 'host_cpu_and_memory.ipynb',
 'type': 'notebook',
 'kernel': {'id': 'cdff474c-25b7-4ed0-bc65-7b6ba587d83e',
  'name': 'python3',
  'last_activity': '2019-03-24T12:45:24.851626Z',
  'execution_state': 'busy',
  'connections': 1},
 'notebook': {'path': 'week_09_timeseries/host_streaming_stats/host_cpu_and_memory.ipynb',
  'name': 'host_cpu_and_memory.ipynb'}}

### from json returned from API, we can extract the notebook kernel IDs

In [44]:
print('notebook kernel IDs:')
for notebook_indx in range(len(list_of_kernel_dicts)):
    print(list_of_kernel_dicts[notebook_indx]['kernel']['id'])

notebook kernel IDs:
cdff474c-25b7-4ed0-bc65-7b6ba587d83e
01a5b1a8-2c46-416e-8241-67d9786046ba


### get a list of all pids
 https://github.com/giampaolo/psutil/issues/132

In [45]:
print('all processes running:')
for p in psutil.process_iter():
    try:
        print(p)
    except psutil.NoSuchProcess:
        pass

all processes running:
psutil.Process(pid=1, name='tini', started='00:07:27')
psutil.Process(pid=7, name='jupyter-lab', started='00:07:27')
psutil.Process(pid=783, name='python', started='12:11:04')
psutil.Process(pid=798, name='python', started='12:45:14')


### get pid for each notebook

In [46]:
def get_all_kernel_pids():
    dict_of_jupyter_kernel_pids = {}
    for p in psutil.process_iter():
        list_of_commands_per_process = psutil.Process(p.pid).cmdline()
#        print(list_of_commands_per_process)
        if ('ipykernel_launcher' in list_of_commands_per_process):
            dict_of_jupyter_kernel_pids[p.pid] = list_of_commands_per_process[len(list_of_commands_per_process)-1]
        
    return dict_of_jupyter_kernel_pids

In [47]:
dict_of_jupyter_kernel_pids = get_all_kernel_pids()
dict_of_jupyter_kernel_pids

{783: '/home/jovyan/.local/share/jupyter/runtime/kernel-01a5b1a8-2c46-416e-8241-67d9786046ba.json',
 798: '/home/jovyan/.local/share/jupyter/runtime/kernel-cdff474c-25b7-4ed0-bc65-7b6ba587d83e.json'}

### (alternative method) use `ps -ax` to get the relevant pid for notebooks

not in use

### add pid to dict containing kernel information

In [48]:
def add_pid_to_kernel_dict(dict_of_jupyter_kernel_pids,list_of_kernel_dicts):
    if not isinstance(dict_of_jupyter_kernel_pids,dict):
        raise TypeError("ERROR: expected a dict as input for dict_of_jupyter_kernel_pids, got",type(dict_of_jupyter_kernel_pids))
        return None
    if not isinstance(list_of_kernel_dicts,list):
        raise TypeError("ERROR: expected a list as input for list_of_kernel_dicts, got",type(list_of_kernel_dicts))
        return None
    for pid,kernel_path in dict_of_jupyter_kernel_pids.items():
        for notebook_indx in range(len(list_of_kernel_dicts)):
            if (list_of_kernel_dicts[notebook_indx]['kernel']['id'] in kernel_path):
#                print(pid,'is',list_of_kernel_dicts[notebook_indx],'\n')
                list_of_kernel_dicts[notebook_indx]['pid']=pid
    for kernel_dict in list_of_kernel_dicts:
        if 'pid' not in kernel_dict.keys():
            print('this kernel_dict is missing a pid:')
            print(kernel_dict)
            print('and the list of PIDs is')
            print(dict_of_jupyter_kernel_pids)
    return list_of_kernel_dicts

In [49]:
list_of_kernel_dicts= add_pid_to_kernel_dict(dict_of_jupyter_kernel_pids,list_of_kernel_dicts)

In [50]:
list_of_kernel_dicts

[{'id': '634f6b06-0d01-4880-8165-8508ac87fe6a',
  'path': 'week_09_timeseries/host_streaming_stats/host_cpu_and_memory.ipynb',
  'name': 'host_cpu_and_memory.ipynb',
  'type': 'notebook',
  'kernel': {'id': 'cdff474c-25b7-4ed0-bc65-7b6ba587d83e',
   'name': 'python3',
   'last_activity': '2019-03-24T12:45:24.851626Z',
   'execution_state': 'busy',
   'connections': 1},
  'notebook': {'path': 'week_09_timeseries/host_streaming_stats/host_cpu_and_memory.ipynb',
   'name': 'host_cpu_and_memory.ipynb'},
  'pid': 798},
 {'id': 'db69d3a6-8727-47ca-987e-a734cea23799',
  'path': 'backblaze/convert_zip_files_to_pkl.ipynb',
  'name': 'convert_zip_files_to_pkl.ipynb',
  'type': 'notebook',
  'kernel': {'id': '01a5b1a8-2c46-416e-8241-67d9786046ba',
   'name': 'python3',
   'last_activity': '2019-03-24T12:11:09.160605Z',
   'execution_state': 'starting',
   'connections': 2},
  'notebook': {'path': 'backblaze/convert_zip_files_to_pkl.ipynb',
   'name': 'convert_zip_files_to_pkl.ipynb'},
  'pid': 78

### which kernels are not this notebook?

In [51]:
def kernel_dicts_other_notebooks(list_of_kernel_dicts):
    if not isinstance(list_of_kernel_dicts,list):
        raise TypeError("ERROR: expected a list as input for list_of_kernel_dicts, got",type(list_of_kernel_dicts))
        return None
    list_of_kernel_dicts_for_other_notebooks=[]
    for kernel_dict in list_of_kernel_dicts:
        if 'pid' in kernel_dict.keys():
            if (str(os.getpid())!=str(kernel_dict['pid'])):
                list_of_kernel_dicts_for_other_notebooks.append(kernel_dict)
        else:
            print('pid not found in key for kernel_dicts')
            print(list_of_kernel_dicts)
    return list_of_kernel_dicts_for_other_notebooks

In [52]:
list_of_kernel_dicts_for_other_notebooks = kernel_dicts_other_notebooks(list_of_kernel_dicts)
list_of_kernel_dicts_for_other_notebooks

[{'id': 'db69d3a6-8727-47ca-987e-a734cea23799',
  'path': 'backblaze/convert_zip_files_to_pkl.ipynb',
  'name': 'convert_zip_files_to_pkl.ipynb',
  'type': 'notebook',
  'kernel': {'id': '01a5b1a8-2c46-416e-8241-67d9786046ba',
   'name': 'python3',
   'last_activity': '2019-03-24T12:11:09.160605Z',
   'execution_state': 'starting',
   'connections': 2},
  'notebook': {'path': 'backblaze/convert_zip_files_to_pkl.ipynb',
   'name': 'convert_zip_files_to_pkl.ipynb'},
  'pid': 783}]

In [53]:
def get_other_kernel_pids(list_of_kernel_dicts_for_other_notebooks,
                          url_for_api_sessions='http://localhost:8888/api/sessions'):
    if not isinstance(list_of_kernel_dicts_for_other_notebooks,list):
        raise TypeError("ERROR: expected a list as input for list_of_kernel_dicts_for_other_notebooks, got",type(list_of_kernel_dicts_for_other_notebooks))
        return None

    list_of_kernel_dicts = requests.get(url_for_api_sessions).json()
    dict_of_jupyter_kernel_pids = get_all_kernel_pids()
    list_of_kernel_dicts= add_pid_to_kernel_dict(dict_of_jupyter_kernel_pids,list_of_kernel_dicts)
    current_list_of_kernel_dicts_for_other_notebooks = kernel_dicts_other_notebooks(list_of_kernel_dicts)

    list_of_pids_in_previous=[]
    for old_dict in list_of_kernel_dicts_for_other_notebooks:
        list_of_pids_in_previous.append(old_dict['pid'])
    for curr_dict in current_list_of_kernel_dicts_for_other_notebooks:
        if (curr_dict['pid'] not in list_of_pids_in_previous):
            list_of_kernel_dicts_for_other_notebooks.append(curr_dict)
    return list_of_kernel_dicts_for_other_notebooks

# collect data for other notebook and create plots

In [54]:
#monitoring_duration_in_seconds_other_notebooks=60 # 60 queries takes 63 seconds

In [55]:
nb_dict={}
list_of_kernel_dicts_for_other_notebooks = get_other_kernel_pids([])
print('there are',len(list_of_kernel_dicts_for_other_notebooks),'other kernels')
prev_kernel_count = len(list_of_kernel_dicts_for_other_notebooks)

start_time = time.time()
for indx in range(monitoring_duration_in_seconds_other_notebooks):
    row = {}
    system_row = row_for_system_performance(row)

    # detect new notebook kernels
    list_of_kernel_dicts_for_other_notebooks = get_other_kernel_pids(list_of_kernel_dicts_for_other_notebooks)
    if len(list_of_kernel_dicts_for_other_notebooks)!=prev_kernel_count:
        print('\nkernel count changed from',prev_kernel_count)
        prev_kernel_count=len(list_of_kernel_dicts_for_other_notebooks)
        print('to',prev_kernel_count)
        print('elapsed time:',time.time()-start_time,'seconds')
        print(list_of_kernel_dicts_for_other_notebooks)
    
    for other_nb in list_of_kernel_dicts_for_other_notebooks:
        row = row_for_performance_per_pid(system_row,int(other_nb['pid']))
        try:
            nb_dict[other_nb['pid']].append(row)
        except KeyError: # new pid for new kernel
            nb_dict[other_nb['pid']] = []
            nb_dict[other_nb['pid']].append(row)
    time.sleep(polling_period_in_seconds)

    #print('elapsed time:',time.time()-start_time,'seconds')
print('elapsed time:',time.time()-start_time,'seconds')

nb_df = {}
for other_nb in list_of_kernel_dicts_for_other_notebooks:
    nb_df[other_nb['pid']] = pandas.DataFrame(nb_dict[other_nb['pid']])

there are 1 other kernels

kernel count changed from 1
to 2
elapsed time: 51.59217977523804 seconds
[{'id': 'db69d3a6-8727-47ca-987e-a734cea23799', 'path': 'backblaze/convert_zip_files_to_pkl.ipynb', 'name': 'convert_zip_files_to_pkl.ipynb', 'type': 'notebook', 'kernel': {'id': '01a5b1a8-2c46-416e-8241-67d9786046ba', 'name': 'python3', 'last_activity': '2019-03-24T12:11:09.160605Z', 'execution_state': 'starting', 'connections': 2}, 'notebook': {'path': 'backblaze/convert_zip_files_to_pkl.ipynb', 'name': 'convert_zip_files_to_pkl.ipynb'}, 'pid': 783}, {'id': '4a4d2267-9e77-4aa9-89c6-474ccb04a3a5', 'path': 'backblaze/quarterly_pkl_to_metrics_pkls.ipynb', 'name': 'quarterly_pkl_to_metrics_pkls.ipynb', 'type': 'notebook', 'kernel': {'id': '3463627e-bb28-460a-a35e-c49986405cbc', 'name': 'python3', 'last_activity': '2019-03-24T12:46:16.696101Z', 'execution_state': 'starting', 'connections': 0}, 'notebook': {'path': 'backblaze/quarterly_pkl_to_metrics_pkls.ipynb', 'name': 'quarterly_pkl_to_me

In [56]:
nb_df.keys()

dict_keys([783, 826, 841, 857])

In [57]:
for indx in range(len(list_of_kernel_dicts_for_other_notebooks)):
    nb_pid = list_of_kernel_dicts_for_other_notebooks[indx]['pid']
    list_of_kernel_dicts_for_other_notebooks[indx]['df']=nb_df[nb_pid]

In [58]:
with open("list_of_df_other_notebooks_"+str(time.time())+".pkl", 'wb') as handle:
    pickle.dump(list_of_kernel_dicts_for_other_notebooks, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('filename.pickle', 'rb') as handle:
#   list_of_kernel_dicts_for_other_notebooks = pickle.load(handle)